## Dataset

https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification

Using pretrained Imagenet model

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)

In [2]:
# Freeze base model
base_model.trainable = False

In [4]:
inputs = keras.Input(shape=(224,224,3))

x = base_model(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(6, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
model.compile(loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [7]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)

In [8]:
# loading the training set
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# loading the validation set
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


In [9]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
37/36 [==============================] - 30s 811ms/step - loss: 3.7741 - accuracy: 0.3985 - val_loss: 1.7582 - val_accuracy: 0.5289
Epoch 2/20
37/36 [==============================] - 19s 507ms/step - loss: 1.1105 - accuracy: 0.7107 - val_loss: 0.7918 - val_accuracy: 0.7690
Epoch 3/20
37/36 [==============================] - 19s 503ms/step - loss: 0.5687 - accuracy: 0.8409 - val_loss: 0.5274 - val_accuracy: 0.8237
Epoch 4/20
37/36 [==============================] - 19s 505ms/step - loss: 0.3966 - accuracy: 0.8824 - val_loss: 0.4821 - val_accuracy: 0.8754
Epoch 5/20
37/36 [==============================] - 19s 503ms/step - loss: 0.2527 - accuracy: 0.9188 - val_loss: 0.3720 - val_accuracy: 0.9088
Epoch 6/20
37/36 [==============================] - 19s 503ms/step - loss: 0.1902 - accuracy: 0.9349 - val_loss: 0.2753 - val_accuracy: 0.9210
Epoch 7/20
37/36 [==============================] - 19s 505ms/step - loss: 0.1378 - accuracy: 0.9560 - val_loss: 0.2190 - val_accuracy: 0.9301

In [10]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 330ms/step - loss: 0.1901 - accuracy: 0.9514


[0.1900564283132553, 0.9513677954673767]